In [1]:
import pandas as pd
from collections import Counter
from glob import glob
import csv
import os.path as path

In [2]:
from pie_extended.cli.utils import get_tagger
from pie_extended.models.dum.imports import get_iterator_and_processor

In [81]:
path_name = 'txt/*.txt'

In [4]:
model_name = "dum"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)

2021-11-28 19:03:34,472 : Model /home/humanum/.local/lib/python3.8/site-packages/pie_extended/downloads/dum/lemma.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.
2021-11-28 19:03:34,996 : Model /home/humanum/.local/lib/python3.8/site-packages/pie_extended/downloads/dum/pos.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.


In [73]:
def dict_freq_tri_pos(list_token, doc_name):
    list_tri_pos = ['n_v_pron', 'pron_n_v', 'adp_pron_n', 'adp_art_n', 'v_pron_n', 'n_adv_v', 'n_adp_n', 'art_n_v', 'adv_v_pron', 'v_pron_v', 'n_pron_n', 'n_n_v', 'n_art_n', 'v_art_n', 'v_adv_v', 'n_v_v', 'n_conj_n', 'pron_v_pron', 'v_conj_pron', 'n_pron_v', 'art_n_n', 'pron_adv_v', 'v_adp_pron', 'n_adp_pron', 'n_conj_pron', 'pron_n_n', 'n_n_n', 'v_adp_n', 'n_v_adv', 'pron_n_pron', 'pron_v_v', 'v_pron_adv', 'art_n_adp', 'pron_n_adv', 'v_v_pron', 'n_v_conj', 'art_adj_n', 'adv_v_v', 'n_v_adp', 'pron_v_adp', 'adp_n_n', 'v_pron_pron', 'adp_n_v', 'adv_v_adv', 'conj_pron_v', 'pron_v_adv', 'adv_adv_v', 'n_n_pron', 'conj_pron_n', 'pron_pron_v']
    dict_result = dict.fromkeys(list_tri_pos)
    
    list_trigram = trigrammize(list_token)

    dict_temp = Counter(list_trigram)
    
    for key in dict_temp.keys():
        if key in dict_result.keys():
            dict_result[key] = dict_temp[key]/len(list_trigram)
    
    return dict_result

In [37]:
def trigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de tri-grammes"""
    list_trigram = []
    for indice_token in range(len(list_token)-2):
        trigram = list_token[indice_token].split("(")[0]+'_'+list_token[indice_token+1].split("(")[0]+'_'+list_token[indice_token+2].split("(")[0]
        list_trigram.append(trigram)
    return list_trigram

In [72]:
def dict_freq_tri_char(list_tri_chars, doc_name):
    list_tri_chars_select = ['en ', 'er ', 'et ', 'n, ', 'en,', 't, ', ' de', 'den', 'an ', 'de ', ' he', 'ijn', ' ge', 'ver', 'een', ' ve', 'at ', 'oor', 'n d', 'aer', ' en', 'jn ', ' da', 'sch', 'eer', 'der', 'ck ', 'cht', 'ie ', ' va', ', D', ' ee', ' be', ' me', 'nde', ' we', 'aar', 'n s', 'ten', ' in', 't d', ' di', 'iet', ' mi', 'in ', 't g', 'ghe', 'te ', ' al', ' En']
    dict_result = dict.fromkeys(list_tri_chars_select)
    
    dict_temp = Counter(list_tri_chars)
    
    for key in dict_temp.keys():
        if key in dict_result.keys():
            dict_result[key] = dict_temp[key]/len(list_tri_chars)
    
    return dict_result

In [39]:
def tri_chars(chaine):
    list_three_chars = []
    for i in range(0, len(chaine)-2):
        list_three_chars.append(chaine[i]+chaine[i+1]+chaine[i+2])   
    return list_three_chars

In [71]:
def dict_freq_lemma(list_lemma, doc_name):
    list_lemma_select = ['een', 'de', 'ik', 'gij', 'hij', 'ne', 'in', 'mijn', 'van', 'het', 'uw', 'en', 'zijn', 'wij', 'zij', 'eten', 'niet', 'dat', 'te', 'met', 'op', 'die', 'haar', 'eer', 'voor', 'hebben', 'zo', 'al', 'als', 'aan', 'et', 'hoofd', 'na', 'tot', 'ook', 'door', 'hier', 'of', 'hoe', 'dan', 'geen', 'om', 'nu', 'wel', 'ons', 'haten', 'deze', 'wat', 'dit', 'bij']
    dict_result = dict.fromkeys(list_lemma_select)
    
    dict_temp = Counter(list_lemma)
    
    for key in dict_temp.keys():
        if key in dict_result.keys():
            dict_result[key] = dict_temp[key]/len(list_lemma)
    
    return dict_result

In [41]:
def nettoie_lemma(list_lemma, list_pos):
    pos_ok = ["pre", "adv", "det", "conj", "pron", "adp", "art"]
    list_nettoie = []
    if len(list_lemma) == len(list_pos):
        for i in range(0, len(list_lemma)-1):
            if list_pos[i].split("(")[0] in pos_ok:
                list_nettoie.append(list_lemma[i])
    return list_nettoie

In [42]:
def main(path_name, doc_name):
    with open(path_name, encoding="utf8") as file:
        lignes = file.readlines()

        for ligne in lignes:
            iterator, processor = get_iterator_and_processor()
            annotation = tagger.tag_str(ligne, iterator=iterator, processor=processor)
            list_three_chars = tri_chars(ligne)

    df_pie = pd.DataFrame(annotation)
    
    dict_result_pos = dict_freq_tri_pos(df_pie['pos'], doc_name)
         
    dict_result_char = dict_freq_tri_char(list_three_chars, doc_name)
    
    list_lemma = nettoie_lemma(df_pie['lemma'], df_pie['pos'])
    dict_result_lemma = dict_freq_lemma(list_lemma, doc_name)

    return dict_result_pos, dict_result_char, dict_result_lemma

In [76]:
def moulinette(path_name):
    dict_results = {}
    
    df_main = pd.DataFrame()
    
    for doc in glob(path_name):
        doc_name = path.splitext(path.basename(doc))[0]
        dict_result_pos, dict_result_char, dict_result_lemma = main(doc, doc_name)
        
        dict_results.update(dict_result_pos)
        dict_results.update(dict_result_char)
        dict_results.update(dict_result_lemma)
        
        dict_results["index"] = doc_name
        
        
        df_temp = pd.DataFrame(dict_results, index=[0])
        
        df_main = df_main.append(df_temp, ignore_index = True)
        
        
    df_main.set_index("index", inplace = True)
    
    return df_main

In [82]:
df_main = moulinette(path_name)

In [83]:
df_main

,n_v_pron,pron_n_v,adp_pron_n,adp_art_n,v_pron_n,n_adv_v,n_adp_n,art_n_v,adv_v_pron,v_pron_v,...,geen,om,nu,wel,ons,haten,deze,wat,dit,bij
index,,,,,,,,,,,,,,,,,,,,,
13732,0.014815,0.007407,0.007407,0.022222,None,0.007407,0.014815,None,None,0.007407,...,0.014706,None,0.014706,None,None,None,0.014706,None,None,None
21533,0.003868,0.017408,0.007737,0.007737,0.023211,0.023211,None,0.005803,0.025145,0.025145,...,0.004032,0.004032,0.008065,0.016129,0.004032,0.008065,0.004032,0.004032,None,None
3542,0.017621,0.004405,None,None,0.004405,0.008811,0.004405,0.013216,None,0.008811,...,None,None,0.009009,None,None,0.009009,None,0.009009,None,None
186454,0.018018,None,0.018018,0.009009,None,None,0.036036,0.009009,0.018018,0.018018,...,0.017857,None,None,None,None,0.017857,0.035714,None,0.053571,0.017857
185051,0.015936,0.007968,0.011952,0.01992,0.007968,0.003984,0.003984,0.011952,0.015936,0.01992,...,0.007752,None,0.007752,None,0.007752,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117156,0.028226,0.016129,0.014113,0.014113,0.022177,0.012097,0.010081,0.006048,0.020161,0.022177,...,0.004065,0.020325,0.004065,0.02439,None,0.01626,0.00813,0.01626,0.00813,None
6213,0.018817,0.013441,0.016129,0.016129,0.010753,0.016129,0.005376,0.013441,0.008065,0.005376,...,0.005495,None,None,0.027473,0.010989,None,0.005495,0.016484,0.005495,0.005495
191412,0.011062,0.017699,0.017699,0.015487,0.022124,0.00885,0.004425,0.002212,0.006637,0.004425,...,0.009091,None,0.009091,None,None,0.004545,None,None,None,None


In [84]:
df_main.to_csv(r'features.csv', index = True)